<img src="film_strip_img.jpg">

# Recommendations for the New Microsoft Movie Studio

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter

%matplotlib inline
sns.set_context('talk')